### Imports

In [1]:
import sys
sys.path.append("llmapslab")
import os
import json
import re
import load_configs
import importlib
importlib.reload(load_configs)
from load_configs import (
    openai_api_key,
    llama_api_key, ai21_api_key, 
    gemini_api_key
)
from openai_client import call_openai_api

### Standardizing message format through LangChain

BaseMessage(Serializable) - > BaseMessage --> SystemMessage, AIMessage, HumanMessage, ChatMessage, FunctionMessage, ToolMessage
                --> BaseMessageChunk --> SystemMessageChunk, AIMessageChunk, HumanMessageChunk, ChatMessageChunk, FunctionMessageChunk, ToolMessageChunk


In [2]:
from langchain_core.messages import (
    SystemMessage, HumanMessage, AIMessage, ChatMessage)

In [3]:
sysmsg = SystemMessage("You are AI assistant for one line answer. Your name is Lisa")
hmsg = HumanMessage("What type of harware is used for quantum computing") 
print(sysmsg)
print(sysmsg.dict()) ###-> they are serializable for saving converting to json etc. 
print()
print(hmsg)

content='You are AI assistant for one line answer. Your name is Lisa'
{'content': 'You are AI assistant for one line answer. Your name is Lisa', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'system', 'name': None, 'id': None}

content='What type of harware is used for quantum computing'


In [4]:
sys_chatmsg = ChatMessage(role="system", content="hhhh")
sys_chatmsg

ChatMessage(content='hhhh', role='system')

FunctionMessage
This represents the result of a function call. In addition to role and content, this message has a name parameter which conveys the name of the function that was called to produce this result.

ToolMessage
This represents the result of a tool call. This is distinct from a FunctionMessage in order to match OpenAI's function and tool message types. In addition to role and content, this message has a tool_call_id parameter which conveys the id of the call to the tool that was called to produce this result.

In [89]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import FunctionMessage, SystemMessage, HumanMessage, AIMessage

# Define the function message
function_message = FunctionMessage(
    content="Calculate revenue for a specific product",
    name="calculate_revenue",
    arguments={
        "product_name": "wireless mouse"
    }
)

# Create the ChatTemplatePrompt with function message
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="I'd like to know the revenue for wireless mouse."),
    function_message,
    AIMessage(content="Calculating the revenue for wireless mouse...")
])

# Invoke the prompt with the necessary context (if any)
prompt_value = chat_prompt.invoke({
    "product_name": "wireless mouse"
})

print(prompt_value)


messages=[SystemMessage(content='You are a helpful assistant.'), HumanMessage(content="I'd like to know the revenue for wireless mouse."), FunctionMessage(content='Calculate revenue for a specific product', name='calculate_revenue', arguments={'product_name': 'wireless mouse'}), AIMessage(content='Calculating the revenue for wireless mouse...')]


In [92]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import ToolMessage, SystemMessage, HumanMessage, AIMessage

# Define the tool message
tool_message = ToolMessage(
    tool_call_id=1,
    content="Retrieve the revenue data",
    artifact=[1, 2, 3, 4],
    name="calculate_revenue",
    arguments={
        "product_name": "wireless mouse"
    }
)

# Create the ChatTemplatePrompt with tool message
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="I'd like to know the revenue for wireless mouse."),
    tool_message,
    AIMessage(content="Calculating the revenue for wireless mouse...")
])

# Invoke the prompt with the necessary context (if any)
prompt_value = chat_prompt.invoke({
    "product_name": "wireless mouse"
})

print(prompt_value)

messages=[SystemMessage(content='You are a helpful assistant.'), HumanMessage(content="I'd like to know the revenue for wireless mouse."), ToolMessage(content='Retrieve the revenue data', name='calculate_revenue', tool_call_id='1', artifact=[1, 2, 3, 4], arguments={'product_name': 'wireless mouse'}), AIMessage(content='Calculating the revenue for wireless mouse...')]


### Langchain Prompt
inserting variables inside the message so that user can only provide input variables  instead of full text

### python string template with some variables

In [5]:
bot_name = ""
sysmsg_template_str = """You are {bot_role} for one line answer. 
Your name is {bot_name}"""
user_input = ""        
hmsg_template_str = """{user_input}""" 

###  Langchain templates 

In [6]:
from langchain_core.prompts import (
    PromptTemplate,
    ChatMessagePromptTemplate,
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,    
    ChatPromptTemplate
)

#### PromptTemplate is generic template - see the argument and methods

In [7]:
str_prompt = PromptTemplate.from_template(
    sysmsg_template_str)
str_prompt
## identifies the input_variables from the template

PromptTemplate(input_variables=['bot_name', 'bot_role'], template='You are {bot_role} for one line answer. \nYour name is {bot_name}')

In [8]:
str_prompt.__dict__

{'name': None,
 'input_variables': ['bot_name', 'bot_role'],
 'optional_variables': [],
 'input_types': {},
 'output_parser': None,
 'partial_variables': {},
 'metadata': None,
 'tags': None,
 'template': 'You are {bot_role} for one line answer. \nYour name is {bot_name}',
 'template_format': 'f-string',
 'validate_template': False}

##### optional_variables are for MessagePlaceholder in ChatPromptTemplate. This should not be displayed here. I have submitted:

https://github.com/langchain-ai/langchain/issues/24884

https://github.com/langchain-ai/langchain/pull/25017

###### partial variables can be upfront defined so that user need not input them repeatedly

In [9]:
str_prompt = PromptTemplate.from_template(
    sysmsg_template_str, partial_variables={
        "bot_role": "AI Assistant"
    })
str_prompt

PromptTemplate(input_variables=['bot_name'], partial_variables={'bot_role': 'AI Assistant'}, template='You are {bot_role} for one line answer. \nYour name is {bot_name}')

##### Another way of defining partial variables

In [10]:
str_prompt = PromptTemplate.from_template(
    sysmsg_template_str)
str_prompt

PromptTemplate(input_variables=['bot_name', 'bot_role'], template='You are {bot_role} for one line answer. \nYour name is {bot_name}')

In [11]:
str_prompt.partial(bot_role= "AI Assistant")

PromptTemplate(input_variables=['bot_name'], partial_variables={'bot_role': 'AI Assistant'}, template='You are {bot_role} for one line answer. \nYour name is {bot_name}')

In [12]:
### parital variable is not working with PromptTemplate
try:
    str_prompt.format(bot_name="Lisa")
except Exception as e:
    print('Key Error:', e)    

Key Error: 'bot_role'


##### parital variable is not working with PromptTemplate

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[28], line 1
----> 1 str_prompt.format(bot_name="Lisa")

File /mnt/d/myDev/llmapps/venv/lib/python3.9/site-packages/langchain_core/prompts/prompt.py:179, in PromptTemplate.format(self, **kwargs)
    170 """Format the prompt with the inputs.
    171 
    172 Args:
   (...)
    176     A formatted string.
    177 """
    178 kwargs = self._merge_partial_and_user_variables(**kwargs)
--> 179 return DEFAULT_FORMATTER_MAPPING[self.template_format](self.template, **kwargs)

File ~/anaconda3/lib/python3.9/string.py:161, in Formatter.format(self, format_string, *args, **kwargs)
    160 def format(self, format_string, /, *args, **kwargs):
--> 161     return self.vformat(format_string, args, kwargs)

File /mnt/d/myDev/llmapps/venv/lib/python3.9/site-packages/langchain_core/utils/formatting.py:31, in StrictFormatter.vformat(self, format_string, args, kwargs)
     26 if len(args) > 0:
     27     raise ValueError(
     28         "No arguments should be provided, "
     29         "everything should be passed as keyword arguments."
     30     )
---> 31 return super().vformat(format_string, args, kwargs)

File ~/anaconda3/lib/python3.9/string.py:165, in Formatter.vformat(self, format_string, args, kwargs)
    163 def vformat(self, format_string, args, kwargs):
    164     used_args = set()
--> 165     result, _ = self._vformat(format_string, args, kwargs, used_args, 2)
    166     self.check_unused_args(used_args, args, kwargs)
    167     return result

File ~/anaconda3/lib/python3.9/string.py:205, in Formatter._vformat(self, format_string, args, kwargs, used_args, recursion_depth, auto_arg_index)
    201     auto_arg_index = False
    203 # given the field_name, find the object it references
    204 #  and the argument it came from
--> 205 obj, arg_used = self.get_field(field_name, args, kwargs)
    206 used_args.add(arg_used)
    208 # do any conversion on the resulting object

File ~/anaconda3/lib/python3.9/string.py:270, in Formatter.get_field(self, field_name, args, kwargs)
    267 def get_field(self, field_name, args, kwargs):
    268     first, rest = _string.formatter_field_name_split(field_name)
--> 270     obj = self.get_value(first, args, kwargs)
    272     # loop through the rest of the field_name, doing
    273     #  getattr or getitem as needed
    274     for is_attr, i in rest:

File ~/anaconda3/lib/python3.9/string.py:227, in Formatter.get_value(self, key, args, kwargs)
    225     return args[key]
    226 else:
--> 227     return kwargs[key]

KeyError: 'bot_role'

##### making the variable not mandatory and also hide

In [13]:
str_prompt.partial(bot_role= "")

PromptTemplate(input_variables=['bot_name'], partial_variables={'bot_role': ''}, template='You are {bot_role} for one line answer. \nYour name is {bot_name}')

##### invoke produces a StringPromptValue class whereas format gives the text from the prompt with variable filled in  

In [14]:
str_prompt.invoke(input={"bot_name": "Lisa", "bot_role": "assistant"})

StringPromptValue(text='You are assistant for one line answer. \nYour name is Lisa')

####  Message Specific Templates for system, human, ai etc. 

In [15]:
prompt = "May the {subject} be with you"
chat_message_prompt = ChatMessagePromptTemplate.from_template(
    role="Jedi", template=prompt
)
chat_message_prompt.format(subject="force")

ChatMessage(content='May the force be with you', role='Jedi')

In [16]:
sysmsg_prompt = SystemMessagePromptTemplate.from_template(
    sysmsg_template_str)
print(sysmsg_prompt)
sysmsg_prompt.format(bot_role="ai assistant", bot_name="Lisa")

prompt=PromptTemplate(input_variables=['bot_name', 'bot_role'], template='You are {bot_role} for one line answer. \nYour name is {bot_name}')


SystemMessage(content='You are ai assistant for one line answer. \nYour name is Lisa')

In [17]:
usrmsg_prompt = HumanMessagePromptTemplate.from_template(hmsg_template_str)
usrmsg_prompt

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], template='{user_input}'))

In [18]:
usrmsg_prompt.format(user_input="what is your name")

HumanMessage(content='what is your name')

In [19]:
usrmsg_prompt.format(**{"user_input": "what is your name"})

HumanMessage(content='what is your name')

In [20]:
usrmsg_prompt.format_messages(**{"user_input": "what is your name"})

[HumanMessage(content='what is your name')]

#### ChatPromptTemplate is for ChatModels 

##### RAW usage
- instead of string, the input are a List[Tuple('role', 'content')]
- Callable, initializttion can be done wiithout .from_template method

In [21]:
chat_prompt = ChatPromptTemplate([
    ('system', sysmsg_template_str),
    ('human', hmsg_template_str),
],
    partial_variables={
        "bot_role": "AI Assistant"
    }
)
chat_prompt

ChatPromptTemplate(input_variables=['bot_name', 'user_input'], partial_variables={'bot_role': 'AI Assistant'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['bot_name', 'bot_role'], template='You are {bot_role} for one line answer. \nYour name is {bot_name}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], template='{user_input}'))])

##### Lets reuse the message template instead of tuple

In [22]:
chat_prompt = ChatPromptTemplate([
    sysmsg_prompt,
    usrmsg_prompt,
],
    partial_variables={
        "bot_role": "AI Assistant",
        'bot_name': 'Lisa'
    }
)
chat_prompt

ChatPromptTemplate(input_variables=['user_input'], partial_variables={'bot_role': 'AI Assistant', 'bot_name': 'Lisa'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['bot_name', 'bot_role'], template='You are {bot_role} for one line answer. \nYour name is {bot_name}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], template='{user_input}'))])

In [23]:
chat_prompt.invoke({'user_input':'hi how r u '})

ChatPromptValue(messages=[SystemMessage(content='You are AI Assistant for one line answer. \nYour name is Lisa'), HumanMessage(content='hi how r u ')])

##### MessagePlaceHolder and optional_variables 

In [24]:
from langchain_core.prompts import MessagesPlaceholder
placeholder_prompt = MessagesPlaceholder("history")
try:
    placeholder_prompt.format_messages() # raises KeyError
except Exception as e:
    print('key error:', e)
placeholder_prompt = MessagesPlaceholder("history", optional=True)
placeholder_prompt.format_messages() # returns empty list []

placeholder_prompt.format_messages(
    history=[
        ("system", "You are an AI assistant."),
        ("human", "Hello!"),
    ]
)

key error: 'history'


[SystemMessage(content='You are an AI assistant.'),
 HumanMessage(content='Hello!')]

##### use MessagePlaceholder in ChatPromptTemplate

In [25]:
chat_prompt = ChatPromptTemplate([
    sysmsg_prompt,
    MessagesPlaceholder("history", optional=True),
    usrmsg_prompt,
    
],
    partial_variables={
        "bot_role": "AI Assistant",
        'bot_name': 'Lisa'
    }
)
chat_prompt

ChatPromptTemplate(input_variables=['user_input'], optional_variables=['history'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'bot_role': 'AI Assistant', 'bot_name': 'Lisa', 'history': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['bot_name', 'bot_role'], template='You are {bot_role} for one line answer. \nYour name is {bot_name}')), MessagesPlaceholder(variable_name='history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], template='{user_input}'))])

In [26]:
chat_prompt.invoke({'user_input':'hi how r u '})

ChatPromptValue(messages=[SystemMessage(content='You are AI Assistant for one line answer. \nYour name is Lisa'), HumanMessage(content='hi how r u ')])

In [27]:
chat_prompt_value = chat_prompt.invoke({
                   "history": [("human", "what's 5 + 2"), ("ai", "5 + 2 is 7")],
                   "user_input": "now multiply that by 4"
               })
chat_prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are AI Assistant for one line answer. \nYour name is Lisa'), HumanMessage(content="what's 5 + 2"), AIMessage(content='5 + 2 is 7'), HumanMessage(content='now multiply that by 4')])

In [28]:
chat_prompt_value.messages

[SystemMessage(content='You are AI Assistant for one line answer. \nYour name is Lisa'),
 HumanMessage(content="what's 5 + 2"),
 AIMessage(content='5 + 2 is 7'),
 HumanMessage(content='now multiply that by 4')]

### Insert Exxample in Prompt 

#### We can build a custom example selector class , see the basic logic 

In [29]:
from langchain_core.example_selectors.base import BaseExampleSelector
# help(BaseExampleSelector)
class CustomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        # This assumes knowledge that part of the input will be a 'text' key
        new_word = input_variables["input"]
        new_word_length = len(new_word)
        # Initialize variables to store the best match and its length difference
        best_match = None
        smallest_diff = float("inf")
        # Iterate through each example
        for example in self.examples:
            # Calculate the length difference with the first word of the example
            current_diff = abs(len(example["input"]) - new_word_length)
            # Update the best match if the current one is closer in length
            if current_diff < smallest_diff:
                smallest_diff = current_diff
                best_match = example
        return [best_match]

#### LangChain semactic similary example selector

In [30]:
# !pip install langchain_chroma
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.example_selectors import (
    SemanticSimilarityExampleSelector, 
    MaxMarginalRelevanceExampleSelector
)

In [31]:
examples = [ ### have to be in list of dictionary format
     ### the keys are  the input_variables or partial_variables of the template
    {"example_user_input": "give me output in streaming format",
     "example_answer": '["My", "name", "is", "Lisa"]' },
    {"example_user_input": "give me output in numerical format",
     "example_answer": '2000' }
    
]
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,    
    OpenAIEmbeddings(openai_api_key=openai_api_key),    
    Chroma, # The VectorStore class that is used to store the embeddings and do a similarity search over.
    k=1,  # The number of examples to produce.
)

In [32]:
example_selector.select_examples({"example_user_input": "stream a one line story"})

[{'example_answer': '["My", "name", "is", "Lisa"]',
  'example_user_input': 'give me output in streaming format'}]

In [33]:
example_selector.select_examples({"example_user_input": "what is 48 percent of 85 million"})

[{'example_answer': '2000',
  'example_user_input': 'give me output in numerical format'}]

#### FewShowPromptTemplate

In [34]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
example_prompt = PromptTemplate(
    # input_variables=["input", "output"],
    template="example_user_input: {example_user_input}\nexample_answer: {example_answer}",
)

similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Consult with the examples before answer:",
    suffix="Input: {user_input}\nOutput:",
    # input_variables=["adjective"],
)
similar_prompt

FewShotPromptTemplate(input_variables=['user_input'], example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7f824f552df0>, k=1, example_keys=None, input_keys=None, vectorstore_kwargs=None), example_prompt=PromptTemplate(input_variables=['example_answer', 'example_user_input'], template='example_user_input: {example_user_input}\nexample_answer: {example_answer}'), suffix='Input: {user_input}\nOutput:', prefix='Consult with the examples before answer:')

In [35]:
similar_prompt.format(user_input="tell me a poem in stream mode" )

'Consult with the examples before answer:\n\nexample_user_input: give me output in streaming format\nexample_answer: ["My", "name", "is", "Lisa"]\n\nInput: tell me a poem in stream mode\nOutput:'

In [36]:
similar_prompt.format(user_input="35 perccent discount on one lakh" )

'Consult with the examples before answer:\n\nexample_user_input: give me output in numerical format\nexample_answer: 2000\n\nInput: 35 perccent discount on one lakh\nOutput:'

#### FewShotChatMessagePromptTemplate

In [37]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

#### Wrong way of using the example_prompt in FewShotChatMessagePromptTemplate

In [38]:
fewshot_chat_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=chat_prompt, ##this prompt is to format the examples and not for users interaction with model
    examples=examples
)
fewshot_chat_prompt

FewShotChatMessagePromptTemplate(examples=[{'example_user_input': 'give me output in streaming format', 'example_answer': '["My", "name", "is", "Lisa"]'}, {'example_user_input': 'give me output in numerical format', 'example_answer': '2000'}], example_prompt=ChatPromptTemplate(input_variables=['user_input'], optional_variables=['history'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'bot_role': 'AI Assistant', 'bot_name': 'Lisa', 'history': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['bot_name', 'bot_role'], template='You are {bot_role} for one line answer. \nYour name is {bot_name}')), MessagesPlaceholder(variable_name='history', optional=True), HumanMessagePrompt

In [39]:
# Earlier we constructed the chat_prompt 
# chat_prompt = ChatPromptTemplate([
#     sysmsg_prompt,
#     MessagesPlaceholder("history", optional=True),
#     usrmsg_prompt,
    
# ],
#     partial_variables={
#         "bot_role": "AI Assistant",
#         'bot_name': 'Lisa'
#     }
# )
# chat_prompt

In [40]:
### can not use it when ther are input variables in the 
try:
    fewshot_chat_prompt.format(user_input="tell me a story")
except Exception as e:
    print('key error:', e)

key error: 'user_input'


#### correct usage of  FewShotChatMessagePromptTemplate 

In [41]:
prompt_for_example = ChatPromptTemplate(
    [ 
        HumanMessagePromptTemplate.from_template("{exapmple_user_input}"), ### equivallent ("human", "exapmple_user_input")
        AIMessagePromptTemplate.from_template("{example_answer}"),        
    ],
    # input_variables=['example_answer', 'exapmple_user_input']
)
prompt_for_example    

ChatPromptTemplate(input_variables=['example_answer', 'exapmple_user_input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['exapmple_user_input'], template='{exapmple_user_input}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['example_answer'], template='{example_answer}'))])

In [45]:
prompt_for_example = ChatPromptTemplate(
    [ 
        ("human", "{example_user_input}"),
        ("ai", "{example_answer}")
            ]
)
prompt_for_example

ChatPromptTemplate(input_variables=['example_answer', 'example_user_input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['example_user_input'], template='{example_user_input}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['example_answer'], template='{example_answer}'))])

In [46]:
fewshow_chat_message_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=prompt_for_example,
    examples=examples
)
fewshow_chat_message_prompt

FewShotChatMessagePromptTemplate(examples=[{'example_user_input': 'give me output in streaming format', 'example_answer': '["My", "name", "is", "Lisa"]'}, {'example_user_input': 'give me output in numerical format', 'example_answer': '2000'}], example_prompt=ChatPromptTemplate(input_variables=['example_answer', 'example_user_input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['example_user_input'], template='{example_user_input}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['example_answer'], template='{example_answer}'))]))

In [47]:
fewshow_chat_message_prompt.format()

'Human: give me output in streaming format\nAI: ["My", "name", "is", "Lisa"]\nHuman: give me output in numerical format\nAI: 2000'

#### Use the fewshow example in the final prompt

In [48]:
final_chat_prompt = ChatPromptTemplate(
    [
        SystemMessage(content="Consult the following Examples:"),
        fewshow_chat_message_prompt,
        HumanMessagePromptTemplate.from_template("{user_input}")
    ]
)
final_chat_prompt

ChatPromptTemplate(input_variables=['user_input'], messages=[SystemMessage(content='Consult the following Examples:'), FewShotChatMessagePromptTemplate(examples=[{'example_user_input': 'give me output in streaming format', 'example_answer': '["My", "name", "is", "Lisa"]'}, {'example_user_input': 'give me output in numerical format', 'example_answer': '2000'}], example_prompt=ChatPromptTemplate(input_variables=['example_answer', 'example_user_input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['example_user_input'], template='{example_user_input}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['example_answer'], template='{example_answer}'))])), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], template='{user_input}'))])

In [49]:
final_chat_prompt.format(user_input='how is the weather')

'System: Consult the following Examples:\nHuman: give me output in streaming format\nAI: ["My", "name", "is", "Lisa"]\nHuman: give me output in numerical format\nAI: 2000\nHuman: how is the weather'

####  Example Selector dynamically in ChatPromptTemplate

In [51]:
### We have already created a example_selector previously but this 
### is another way to do it
# to_vectorize = [" ".join(example.values()) for example in examples]
# embeddings = OpenAIEmbeddings()
# example_selector = SemanticSimilarityExampleSelector(
#     vectorstore=vectorstore,
#     k=2,
# )
example_selector

SemanticSimilarityExampleSelector(vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7f824f552df0>, k=1, example_keys=None, input_keys=None, vectorstore_kwargs=None)

In [55]:
dynamic_fewshow_chat_message_prompt = FewShotChatMessagePromptTemplate(
    # input_variables=['example_user_input'],
    example_prompt=prompt_for_example,
    example_selector=example_selector
)
dynamic_fewshow_chat_message_prompt

FewShotChatMessagePromptTemplate(example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7f824f552df0>, k=1, example_keys=None, input_keys=None, vectorstore_kwargs=None), example_prompt=ChatPromptTemplate(input_variables=['example_answer', 'example_user_input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['example_user_input'], template='{example_user_input}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['example_answer'], template='{example_answer}'))]))

In [56]:
dynamic_fewshow_chat_message_prompt.format(user_input='what is the temparature')

'Human: give me output in numerical format\nAI: 2000'

In [57]:
dynamic_fewshow_chat_message_prompt.format(user_input='who was the king')

'Human: give me output in streaming format\nAI: ["My", "name", "is", "Lisa"]'

In [59]:
example_selector.add_example({
    "example_user_input": "sing me a song",
    "example_answer": "Chord A , Chord C, ....Chord D"
})

'a96c9014-1f39-4ee8-be97-9a1c73a4307c'

In [60]:
dynamic_fewshow_chat_message_prompt.format(user_input='can you sing jingle bell')

'Human: sing me a song\nAI: Chord A , Chord C, ....Chord D'

### Partial Prompt - from function

In [64]:
from datetime import datetime


def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")

In [65]:
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective", "date"],
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))

Tell me a funny joke about the day 08/04/2024, 18:49:05


In [66]:
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective"],
    partial_variables={"date": _get_datetime},
)
print(prompt.format(adjective="funny"))

Tell me a funny joke about the day 08/04/2024, 18:49:20


### Compositions

In [68]:
prompt = (
    PromptTemplate.from_template("Tell me a joke about {topic}")
    + ", make it funny"
    + "\n\nand in {language}"
)
prompt

PromptTemplate(input_variables=['language', 'topic'], template='Tell me a joke about {topic}, make it funny\n\nand in {language}')

In [69]:
prompt = SystemMessage(content="You are a nice pirate")
new_prompt = (
    prompt + HumanMessage(content="hi") + AIMessage(content="what?") + "{input}"
)
new_prompt.format_messages(input="i said hi")

[SystemMessage(content='You are a nice pirate'),
 HumanMessage(content='hi'),
 AIMessage(content='what?'),
 HumanMessage(content='i said hi')]

### PipeinePromptTemplate

In [72]:
from langchain_core.prompts.pipeline import PipelinePromptTemplate

In [75]:
full_template = """
{introduction}

{example}

{start_action}
"""
full_prompt = PromptTemplate.from_template(full_template)

In [79]:
introduction_prompt = PromptTemplate.from_template(
    "introduction for {subject}")
example_prompt = PromptTemplate.from_template(
    """follow the example:
    Question: what is  the application of Low Rank matrix in {example_subject}
    Answer: " efficient calculation and reduction of GPU RAM usage"""
    )
start_action_prompt = PromptTemplate.from_template(
    "You need to now initiate {action_item}"
)


In [81]:
prompt_list = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start_action", start_action_prompt)
]

In [86]:
final_template = PipelinePromptTemplate(
    final_prompt=full_prompt,
    pipeline_prompts=prompt_list
)
final_template

PipelinePromptTemplate(input_variables=['subject', 'example_subject', 'action_item'], final_prompt=PromptTemplate(input_variables=['example', 'introduction', 'start_action'], template='\n{introduction}\n\n{example}\n\n{start_action}\n'), pipeline_prompts=[('introduction', PromptTemplate(input_variables=['subject'], template='introduction for {subject}')), ('example', PromptTemplate(input_variables=['example_subject'], template='follow the example:\n    Question: what is  the application of Low Rank matrix in {example_subject}\n    Answer: " efficient calculation and reduction of GPU RAM usage')), ('start_action', PromptTemplate(input_variables=['action_item'], template='You need to now initiate {action_item}'))])

In [87]:
final_template.format(
    subject="Artificial Intelligence",
    example_subject="Machine Learning",
    action_item="draw matrix"
)

'\nintroduction for Artificial Intelligence\n\nfollow the example:\n    Question: what is  the application of Low Rank matrix in Machine Learning\n    Answer: " efficient calculation and reduction of GPU RAM usage\n\nYou need to now initiate draw matrix\n'